<a href="https://colab.research.google.com/github/Falconwatch/BankingBattle/blob/main/Investments/HW2/Investments_HW2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from os import listdir
from os.path import isfile, join
import tqdm
import pandas as pd
import polars as ps
from zipfile import ZipFile
import gc
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, time, timedelta
from google.colab import drive

In [2]:
drive.mount('/content/drive')
path_to_folder_with_files = "/content/drive/MyDrive/NES/investments/"

Mounted at /content/drive


# Необходимые переменные для будущих частей

In [3]:
#количество файлов, для обработки (будет обработано не более чем)
FILES_TO_PROCESS=100

In [4]:
blue_chips = ["CHMF", "GAZP", "GMKN", "IRAO", "LKOH", "MGNT", "MTSS", "NVTK", "PLZL", "ROSN", "RUAL","SEBR","SNGS","TATN", "YNDX"]
blue_chips_data = {}

# Первая часть - изучение данных

In [5]:
onlyfiles = [f for f in listdir(path_to_folder_with_files) if isfile(join(path_to_folder_with_files, f))]
datas = {f[10:-4]:{"file":f} for f in onlyfiles[:FILES_TO_PROCESS]}

## Explore data
To explore the data, in a table for each day for each field summarize relevant information,
for example, type of the field, min value, max value, number of NA and missing values etc.

In [6]:
def read_one_day(filename):
  oneday = ps.read_csv(ZipFile(path_to_folder_with_files+filename).read(filename[:-3]+"txt"),
                 null_values=["-"])
  gc.collect()
  return oneday

In [7]:
# one_day_example = read_one_day(datas["231005"]["file"])
# one_day_example

## Filtered obs
In one consolidated table summarize information on filtered out observations for each
type of errors (missing data, NAs, duplicate, bad data etc.) for each data field: how many
observations were deleted due to these errors.

In [8]:
def get_int_columns_stats(df, column):

  if df[column].dtype.is_numeric():
    column_stats = df.select(
         ps.when(ps.col(column).is_nan()).then(1).otherwise(0).sum().alias("nans"),
         ps.when(ps.col(column).is_null()).then(1).otherwise(0).sum().alias("nulls"),
         ps.when(ps.col(column)<0).then(1).otherwise(0).sum().alias("bellow_zero"),
         ps.when(ps.col(column)>0).then(1).otherwise(0).sum().alias("above_zero"),
         ps.when(ps.col(column)==0).then(1).otherwise(0).sum().alias("equal_zero"))
  else:
    column_stats = df.select(
          ps.when(ps.col(column).is_null()).then(1).otherwise(0).sum().alias("nulls"))
  return column_stats.to_dicts()[0]

def describe_bad_data(df, date):
  price_stats = get_int_columns_stats(df, "PRICE")
  volume_stats = get_int_columns_stats(df, "VOLUME")
  time_stats = get_int_columns_stats(df, "TIME")
  seccode_stats = get_int_columns_stats(df, "SECCODE")
  buysell_stats = get_int_columns_stats(df, "BUYSELL")
  tradeno_stats = get_int_columns_stats(df, "TRADENO")

  if "cleaning" not in datas[date]:
    datas[date]["cleaning"]={}
  datas[date]["cleaning"]["price"] = price_stats
  datas[date]["cleaning"]["volume"] = volume_stats
  datas[date]["cleaning"]["time"] = time_stats
  datas[date]["cleaning"]["seccode"] = seccode_stats
  datas[date]["cleaning"]["buysell"] = buysell_stats
  datas[date]["cleaning"]["tradeno"] = tradeno_stats



#Excludes bad observations, for example, transactions with price or size of 0.
def clean_zeros(df, date):
  df_ok = df.filter((ps.col("PRICE")>0) & (ps.col("VOLUME")>0))
  deleted_zeros = df.shape[0] - df_ok.shape[0]
  if "cleaning" not in datas[date]:
    datas[date]["cleaning"]={}
  datas[date]["cleaning"]["deleted_zeros"] = deleted_zeros
  return df_ok


#Excludes observations out of main trading hours 9:50–18:50.
def clear_time(df, date):
  df_ok = df.filter((ps.col("TIME")>=95000000000) & (ps.col("TIME")<=185000000000))
  out_of_time = df.shape[0] - df_ok.shape[0]
  if "cleaning" not in datas[date]:
    datas[date]["cleaning"]={}
  datas[date]["cleaning"]["out_of_time"] = out_of_time
  return df_ok


# Deals with NAs in the data if any.
def clear_nas(df, date,
              columns_to_check = ['SECCODE', 'BUYSELL', 'TIME', 'PRICE', 'VOLUME']):
  df_ok = df.drop_nulls(subset=columns_to_check)
  #df_ok = df.drop_nulls()
  dropped_nans = df.shape[0] - df_ok.shape[0]
  if "cleaning" not in datas[date]:
    datas[date]["cleaning"]={}
  datas[date]["cleaning"]["dropped_nans"] = dropped_nans
  return df_ok

#If there are any duplicates, exclude them as well.
def clean_duplicates(df, date):
  df_ok = df.unique(subset=df.columns)
  dropped_duplicates = df.shape[0] - df_ok.shape[0]
  if "cleaning" not in datas[date]:
    datas[date]["cleaning"]={}
  datas[date]["cleaning"]["dropped_duplicates"] = dropped_duplicates
  return df_ok

In [9]:
for date in tqdm.tqdm(datas.keys()):

  oneday_df = read_one_day(datas[date]["file"])
  #собираю статистику
  describe_bad_data(oneday_df, date)

  oneday_df = clean_zeros(oneday_df, date) # удаляю нулевые транзакции
  oneday_df = clear_nas(oneday_df, date) # удаляю где есть пустоты
  oneday_df = clear_time(oneday_df, date) # удаляю вне времени торговли
  oneday_df = clean_duplicates(oneday_df, date) # удаляю дубликаты
  blue_chips_data[date] = oneday_df.filter(ps.col("SECCODE").is_in(blue_chips)) #сохраняю данные для второй части
  del oneday_df
  gc.collect()


100%|██████████| 22/22 [11:47<00:00, 32.17s/it]


In [10]:
all_stats = [(k, v["cleaning"]) for k,v in datas.items()]
for i in range(len(all_stats)):
  all_stats[i][1]["date"] = all_stats[i][0]
all_stats = [a[1] for a in all_stats]

all_stats_pd = pd.json_normalize(all_stats)

In [11]:
all_stats_pd

,deleted_zeros,dropped_nans,out_of_time,dropped_duplicates,date,price.nans,price.nulls,price.bellow_zero,price.above_zero,price.equal_zero,...,time.bellow_zero,time.above_zero,time.equal_zero,seccode.nulls,buysell.nulls,tradeno.nans,tradeno.nulls,tradeno.bellow_zero,tradeno.above_zero,tradeno.equal_zero
0,787,0,11642,771,231005,0,787,0,17611206,0,...,0,17611993,0,0,0,0,12234044,0,5377949,0
1,6722,0,19868,1260,231013,0,6722,0,17705095,0,...,0,17711817,0,0,0,0,11883185,0,5828632,0
2,1949,0,17744,2650,231020,0,1949,0,18128944,0,...,0,18130893,0,0,0,0,11776417,0,6354476,0
3,3557,0,18507,1381,231019,0,3557,0,18149038,0,...,0,18152595,0,0,0,0,11768488,0,6384107,0
4,5686,0,12854,1344,231027,0,5686,0,24245841,0,...,0,24251527,0,0,0,0,17511908,0,6739619,0
5,970,0,13333,1198,231006,0,970,0,17743958,0,...,0,17744928,0,0,0,0,12024600,0,5720328,0
6,982,0,13191,1325,231010,0,982,0,18498625,0,...,0,18499607,0,0,0,0,12422058,0,6077549,0
7,2436,0,15292,1367,231025,0,2436,0,18387866,0,...,0,18390302,0,0,0,0,12186835,0,6203467,0
8,3228,0,17063,1767,231026,0,3228,0,24317072,0,...,0,24320300,0,0,0,0,17217224,0,7103076,0
9,1549,0,19333,1717,231004,0,1549,0,18698716,0,...,0,18700265,0,0,0,0,12978769,0,5721496,0


# Analysis of blue-chip stocks


In [12]:
for k in blue_chips_data.keys():
  blue_chips_data[k] = blue_chips_data[k].with_columns(ps.lit(k).alias("date"))
all_blue_chips_data = ps.concat(blue_chips_data.values())

In [13]:
all_blue_chips_trades = all_blue_chips_data.filter(ps.col("TRADENO").is_not_null())\
.filter(ps.col("BUYSELL")=="S")

In [14]:
all_blue_chips_data\
  .filter((ps.col("SECCODE")=="NVTK") & (ps.col("date")=="231013"))\
  .filter(ps.col("TRADENO").is_not_null())\
.sort("TIME").group_by("BUYSELL").agg(ps.col("VOLUME").sum())

BUYSELL,VOLUME
str,i64
"""S""",374268
"""B""",374268


In [15]:
def find_daily_volatility(group_series):
  group_df = ps.DataFrame(group_series, schema=["TIME", "PRICE"])

  agg = group_df\
    .sort(["TIME"])\
    .with_columns(
        (ps.col("PRICE")/ps.col("PRICE").shift()).log().pow(2).alias("p_diff_log_sq")
        )\
    .select(
        ps.col("p_diff_log_sq").sum().pow(0.5).alias("daily_volatility"),
        )
  return agg["daily_volatility"][0]

def find_quotes_changes(group_series):
  group_df = ps.DataFrame(group_series, schema=["TIME", "PRICE"])

  agg = group_df\
    .sort(["TIME"])\
    .with_columns(
        (ps.col("PRICE")/ps.col("PRICE").shift()).alias("p_diff")
        )\
    .select(
        ps.when(ps.col("p_diff")==1).then(0).otherwise(1).sum().alias("quotes_changes")
        )
  return agg["quotes_changes"][0]

def calc_daily_stats(df):
  daily_stats = df.group_by(["SECCODE", "date"])\
  .agg(
    (ps.col("VOLUME")*ps.col("PRICE")).sum().alias("daily_total_rubble_volume"),
    (ps.col("VOLUME")*ps.col("PRICE")).mean().alias("daily_avg_rubble_volume"),
    (ps.col("VOLUME")*ps.col("PRICE")).median().alias("daily_median_rubble_volume"),

    ps.col("VOLUME").sum().alias("daily_total_volume"),
    ps.col("VOLUME").mean().alias("daily_avg_volume"),
    ps.col("VOLUME").median().alias("daily_median_volume"),

    ps.col("PRICE").mean().alias("daily_avg_price"),
    ps.col("TRADENO").count().alias("daily_transactions"),

    ps.map_groups(
        exprs=[ "TIME", "PRICE"],
        function=find_daily_volatility).alias("daily_volatility"),
    ps.map_groups(
        exprs=[ "TIME", "PRICE"],
        function=find_quotes_changes).alias("daily_quotes_changes"),
  )\
  .with_columns((ps.col("daily_total_rubble_volume")/ps.col("daily_total_volume")).alias("daily_avg_weighted_price"))\
  .with_columns(ps.col("daily_avg_weighted_price").alias("P"),
                ps.col("daily_total_volume").alias("V")
                )\
  .sort(["SECCODE", "date"])
  return daily_stats


daily_stats_TRADES = calc_daily_stats(all_blue_chips_trades)

## 5. Answer the following questions:
a) Which stock was the most/least volatile during the period under investigation?

b) Which stock had the largest/lowest share volume? ruble volume?

c) Which stock had the largest/lowest number of transactions?

d) Which stock had the largest/lowest number of quotes changes?

e) On which day of the week stocks had the largest/lowest volume in the sample?

f) Which stock had the highest/lowest ruble bid-ask spread? percentage bid-ask
spread?

### a) Which stock was the most/least volatile during the period under investigation?

In [16]:
#Which stock was the most/least volatile during the period under investigation?
mean_daily_volatility = daily_stats_TRADES.group_by("SECCODE").agg(ps.col("daily_volatility").mean().alias("mean_daily_volatility"))\
.sort("mean_daily_volatility")
print(mean_daily_volatility[0])
print(mean_daily_volatility[-1])

shape: (1, 2)
┌─────────┬───────────────────────┐
│ SECCODE ┆ mean_daily_volatility │
│ ---     ┆ ---                   │
│ str     ┆ f64                   │
╞═════════╪═══════════════════════╡
│ GMKN    ┆ 0.025073              │
└─────────┴───────────────────────┘
shape: (1, 2)
┌─────────┬───────────────────────┐
│ SECCODE ┆ mean_daily_volatility │
│ ---     ┆ ---                   │
│ str     ┆ f64                   │
╞═════════╪═══════════════════════╡
│ YNDX    ┆ 0.05363               │
└─────────┴───────────────────────┘


### b) Which stock had the largest/lowest share volume? ruble volume?

In [17]:
# Which stock had the largest/lowest share volume? ruble volume?
volumes = daily_stats_TRADES.group_by("SECCODE")\
  .agg(
        ps.col("daily_total_volume").sum().alias("total_volume"),
        ps.col("daily_total_rubble_volume").sum().alias("total_rubble_volume")
      )
#share volume
print("share volume")
print(volumes.sort("total_volume").select("total_volume")[0])
print(volumes.sort("total_volume").select("total_volume")[-1])
print("####################################################")

#ruble volume
print("ruble volume")
print(volumes.sort("total_rubble_volume").select("total_rubble_volume")[0])
print(volumes.sort("total_rubble_volume").select("total_rubble_volume")[-1])

share volume
shape: (1, 1)
┌──────────────┐
│ total_volume │
│ ---          │
│ i64          │
╞══════════════╡
│ 1294065      │
└──────────────┘
shape: (1, 1)
┌──────────────┐
│ total_volume │
│ ---          │
│ i64          │
╞══════════════╡
│ 2414519200   │
└──────────────┘
####################################################
ruble volume
shape: (1, 1)
┌─────────────────────┐
│ total_rubble_volume │
│ ---                 │
│ f64                 │
╞═════════════════════╡
│ 8.9012e9            │
└─────────────────────┘
shape: (1, 1)
┌─────────────────────┐
│ total_rubble_volume │
│ ---                 │
│ f64                 │
╞═════════════════════╡
│ 1.9346e11           │
└─────────────────────┘


### c) Which stock had the largest/lowest number of transactions?

In [18]:
# c) Which stock had the largest/lowest number of transactions?
transactions = daily_stats_TRADES.group_by("SECCODE")\
  .agg(
        ps.col("daily_transactions").sum().alias("total_transactions"),
      )
print(transactions.sort("total_transactions")[0])
print(transactions.sort("total_transactions")[-1])

shape: (1, 2)
┌─────────┬────────────────────┐
│ SECCODE ┆ total_transactions │
│ ---     ┆ ---                │
│ str     ┆ u32                │
╞═════════╪════════════════════╡
│ PLZL    ┆ 218637             │
└─────────┴────────────────────┘
shape: (1, 2)
┌─────────┬────────────────────┐
│ SECCODE ┆ total_transactions │
│ ---     ┆ ---                │
│ str     ┆ u32                │
╞═════════╪════════════════════╡
│ LKOH    ┆ 1574648            │
└─────────┴────────────────────┘


### d) Which stock had the largest/lowest number of quotes changes?

In [19]:

# d) Which stock had the largest/lowest number of quotes changes?
quotes_changes = daily_stats_TRADES.group_by("SECCODE")\
  .agg(
        ps.col("daily_quotes_changes").sum().alias("total_quotes_changes"),
      )
print(quotes_changes.sort("total_quotes_changes")[0])
print(quotes_changes.sort("total_quotes_changes")[-1])

shape: (1, 2)
┌─────────┬──────────────────────┐
│ SECCODE ┆ total_quotes_changes │
│ ---     ┆ ---                  │
│ str     ┆ i64                  │
╞═════════╪══════════════════════╡
│ MTSS    ┆ 107436               │
└─────────┴──────────────────────┘
shape: (1, 2)
┌─────────┬──────────────────────┐
│ SECCODE ┆ total_quotes_changes │
│ ---     ┆ ---                  │
│ str     ┆ i64                  │
╞═════════╪══════════════════════╡
│ LKOH    ┆ 729430               │
└─────────┴──────────────────────┘


### e) On which day of the week stocks had the largest/lowest volume in the sample?

In [20]:
# e) On which day of the week stocks had the largest/lowest volume in the sample?
volume_on_date = daily_stats_TRADES.group_by("date")\
  .agg(ps.col("daily_total_rubble_volume").sum().alias("date_total_rubble_volume"),
       ps.col("daily_total_volume").sum().alias("date_total_volume"))
volume_on_date = volume_on_date.with_columns(ps.col("date").str.to_date(format="%y%m%d").dt.weekday().alias("weekday"))

volume_on_wd = volume_on_date.group_by("weekday")\
  .agg(
      ps.col("date_total_volume").sum().alias("weekday_total_volume"),
      ps.col("date_total_rubble_volume").sum().alias("weekday_total_rubble_volume"),
                                       )
volume_on_wd


#share volume
print("weekday volume")
print(volume_on_wd.sort("weekday_total_volume").select("weekday_total_volume")[0])
print(volume_on_wd.sort("weekday_total_volume").select("weekday_total_volume")[-1])
print("####################################################")

#ruble volume
print("weekday ruble volume")
print(volume_on_wd.sort("weekday_total_rubble_volume").select("weekday_total_rubble_volume")[0])
print(volume_on_wd.sort("weekday_total_rubble_volume").select("weekday_total_rubble_volume")[-1])

weekday volume
shape: (1, 1)
┌──────────────────────┐
│ weekday_total_volume │
│ ---                  │
│ i64                  │
╞══════════════════════╡
│ 568231989            │
└──────────────────────┘
shape: (1, 1)
┌──────────────────────┐
│ weekday_total_volume │
│ ---                  │
│ i64                  │
╞══════════════════════╡
│ 1344414480           │
└──────────────────────┘
####################################################
weekday ruble volume
shape: (1, 1)
┌─────────────────────────────┐
│ weekday_total_rubble_volume │
│ ---                         │
│ f64                         │
╞═════════════════════════════╡
│ 1.1130e11                   │
└─────────────────────────────┘
shape: (1, 1)
┌─────────────────────────────┐
│ weekday_total_rubble_volume │
│ ---                         │
│ f64                         │
╞═════════════════════════════╡
│ 1.3167e11                   │
└─────────────────────────────┘


### f) Which stock had the highest/lowest ruble bid-ask spread? percentage bid-ask spread?

In [21]:
# spreads = all_blue_chips_data\
# .filter(ps.col("TRADENO").is_null())\
# .with_columns(ps.when(ps.col("BUYSELL")=="B").then(ps.col("PRICE")).otherwise(None).alias("buy_price"),
#               ps.when(ps.col("BUYSELL")=="S").then(ps.col("PRICE")).otherwise(None).alias("sell_price"))\
# .group_by("SECCODE", "date")\
# .agg(ps.col("buy_price").min().alias("best_buy"),
#      ps.col("sell_price").max().alias("best_sell"))\
# .with_columns((ps.col("best_sell") - ps.col("best_buy")).alias("spread"))\
# .with_columns((100*ps.col("spread")/ps.col("best_sell")).alias("spread_perc"))

In [ ]:
# f) Which stock had the highest/lowest ruble bid-ask spread? percentage bid-ask spread?
spreads = all_blue_chips_data\
.filter(ps.col("TRADENO").is_null())\
.with_columns(ps.when(ps.col("BUYSELL")=="B").then(ps.col("PRICE")).otherwise(None).alias("buy_price"),
              ps.when(ps.col("BUYSELL")=="S").then(ps.col("PRICE")).otherwise(None).alias("sell_price"))\
.group_by("SECCODE", "date", "TIME")\
.agg(ps.col("buy_price").max().alias("best_buy"),
     ps.col("sell_price").min().alias("best_sell"))\
.with_columns((ps.col("best_sell") - ps.col("best_buy")).alias("spread"))\
.with_columns((100*ps.col("spread")/(ps.col("best_sell") + ps.col("best_buy"))/2).alias("spread_perc"))\
.with_columns((ps.col("spread")>0).alias("spread_positive"))\
.filter(ps.col("spread").is_not_null())\
.filter(ps.col("spread_positive")==True)\

spreads.group_by("SECCODE").agg(ps.col("spread").max())\
.sort("spread", descending=True)[:3]

In [ ]:
spreads.group_by("SECCODE").agg(ps.col("spread_perc").max())\
.sort("spread_perc", descending=True)[:3]

## 6) Examine empirically what is the relationship between the number of trades and thenumber of quotes changes? For example, plot the log of the number of trades against the log of the number of quotes changes calculated for each stock and each trading day.

In [ ]:
trades_count = all_blue_chips_trades\
.group_by("date", "SECCODE")\
.agg(ps.col("TRADENO").n_unique().log().alias("trades_count_log"))

daily_quotes_changes = daily_stats_TRADES.select("SECCODE", "date",
                                              ps.col("daily_quotes_changes").log().alias("daily_quotes_changes_log"))
changes_vs_trades = trades_count.join(daily_quotes_changes, on =["SECCODE", "date"])

sns.scatterplot(data = changes_vs_trades, x="trades_count_log", y="daily_quotes_changes_log")

## 7) Trading volume often exhibits specific intraday patterns. Examine intraday patterns in volume by calculating the average ruble volume executed during each 30-min period between 9:50 and 18:50; see figure 10 in example. Stocks differ a lot by volume. To aggregate data across 15 stocks into one chart, you may plot, for example, the typical fraction of daily volume executed during each thirty-minute interval (and then averaged across all stocks). How would you suggest to treat the first and the last intervals given that they are shorter than the rest of intervals?

In [ ]:
def str_to_time(examp):
  micro = int(examp[-6:])
  ss = int(examp[-8:-6])
  mm = int(examp[-10:-8])
  hh = int(examp[:-10])
  delta = timedelta(hours=hh, minutes=mm, seconds=ss, microseconds=micro)
  return delta.total_seconds()


In [ ]:
str_to_time("101000000000")//1800

In [ ]:
all_blue_chips_data = all_blue_chips_data.with_columns(ps.col("TIME").map_elements(lambda x: str_to_time(str(x)),
                                                             return_dtype=float).alias("TIME_secs"))

In [ ]:
data_for_hh_analysis =\
all_blue_chips_trades\
.with_columns((ps.col("TIME_secs") // 1800).alias("hh"))\
.group_by("date", "hh", "SECCODE")\
.agg(ps.col("VOLUME").sum().alias("hh_volume"))\
.sort("date", "hh", "SECCODE")

data_for_hh_analysis_daily =\
data_for_hh_analysis\
.group_by("date","SECCODE")\
.agg(ps.col("hh_volume").sum().alias("hh_volume_daily"))

data_for_hh_analysis_predfinal = data_for_hh_analysis.join(data_for_hh_analysis_daily,
                                                       on=["date", "SECCODE"])\
.with_columns((ps.col("hh_volume")/ps.col("hh_volume_daily")).alias("hh_fraction"))
data_for_hh_analysis_predfinal.head()

In [ ]:
def hh_to_str(hh):
  hour = int(hh*1800/60/60)
  if hh%1 == 0:
    return f"{hour}:00 - {hour}:29"
  else:
    return f"{hour}:30 - {hour}:59"



hh_fraction_for_plot = data_for_hh_analysis_predfinal.group_by("hh")\
.agg(ps.col("hh_fraction").mean().alias("fraction"))\
.sort("hh")

hh_fraction_for_plot = hh_fraction_for_plot.with_columns(ps.col("hh")\
                                                         .map_elements(lambda x: hh_to_str(x)).alias("hh_str"))
ax = hh_fraction_for_plot.to_pandas().set_index("hh").drop("hh_str", axis=1).plot(figsize=[10,7])
_ = ax.set_xticks(hh_fraction_for_plot["hh"])
_ = ax.set_xticklabels(hh_fraction_for_plot["hh_str"], rotation=45)
#ax.set_xticks(hh_fraction_for_plot["hh_str"],)

# Test of invariance predictions

## 8) Examine whether bid-ask spreads for the set of blue-chip stocks are consistent with invariance predictions.

### a) Plot ln(Spread/P) vs ln(1/L). Check whether the slope is similar to 1 as predicted by invariance. Do you observe any interesting deviations from predictions? How to explain them?

### b) Color observations for different stocks in different colors (15 colors in total). Do you see any interesting patterns?

In [ ]:
daily_stats_CUT = daily_stats_TRADES\
.select("SECCODE", "date", "P", "V", "daily_volatility")\
.with_columns(
    (daily_stats_TRADES["P"]*daily_stats_TRADES["V"]/daily_stats_TRADES["daily_volatility"].pow(2)).pow(-1/3).log().alias("log(1/L)"))

spreads_daily = spreads.group_by("SECCODE", "date").agg(ps.col("spread").max())

In [ ]:
daily_stats_CUT = daily_stats_CUT.join(spreads_daily, on=["SECCODE", "date"])\
.with_columns((ps.col("spread")/ps.col("P")).alias("S/P"))\
.with_columns(ps.col("S/P").log().alias("log(S/P)"))
daily_stats_CUT.head()

In [ ]:
ax = sns.scatterplot(data = daily_stats_CUT, x="log(1/L)", y = "log(S/P)", hue="SECCODE")
sns.regplot(data=daily_stats_CUT, x="log(1/L)", y="log(S/P)", scatter=False, ax=ax, )

### c) Plot ln(Spread) vs ln(P/L). Color observations for different stocks in different colors (15 colors in total). Do you see any interesting patterns? How to explain them?

In [ ]:
daily_stats_CUT = daily_stats_CUT.with_columns(
    ps.col("spread").log().alias("log(spread)"),
    (ps.col("P").log() + ps.col("log(1/L)")).alias("log(P/L)"))
daily_stats_CUT.head()

In [ ]:
ax = sns.scatterplot(data = daily_stats_CUT, x="log(P/L)", y = "log(spread)", hue="SECCODE", legend=False)
sns.regplot(data=daily_stats_CUT, x="log(P/L)", y="log(spread)", scatter=False, ax=ax, )
ax.legend(loc='lower right', )

## 9) [Extra*] For each blue-chip stock try to infer its tick size from the data. Check whether patterns in 5c) are consistent with existence of institutional restriction on minimal price change.

In [ ]:
all_blue_chips_trades = all_blue_chips_data.filter(ps.col("TRADENO").is_not_null())\
.filter(ps.col("BUYSELL")=="S")

In [ ]:
all_blue_chips_trades = all_blue_chips_trades\
.with_columns(
    ps.col("PRICE").sort_by("TIME").shift().over("SECCODE", "date").alias("prev_price"))\
.with_columns(
    (ps.col("PRICE") - ps.col("prev_price")).abs().alias("price_diff_abs"))

price_diffs_stats = all_blue_chips_trades.group_by("SECCODE", "price_diff_abs")\
.agg(ps.col("TRADENO").count().alias("count"))\
.filter(ps.col("price_diff_abs")>0)

In [ ]:
def gcd_array(arr):
    result = arr[0]
    for i in range(1, len(arr)):
        result = np.gcd(result, arr[i])
    return result
TMP = None
def gcd_udf(group_series):
  т = 8
  multiplier = 10**n
  values_arr = (np.round((group_series[0]).to_list(), n) * multiplier).astype(np.int64)
  result = gcd_array(values_arr)/multiplier
  return result


price_tick_sizes = price_diffs_stats\
.group_by("SECCODE").agg(
  ps.map_groups(
        exprs=["price_diff_abs"],
        function=gcd_udf).alias("abc"))

#Analysis of a simple trading strategy

## 10) Construct the following strategy for 15 blue-chip stocks.

At the beginning of each hour, for each blue-chip stock check imbalance at the best bid
and best offer and then trade into the direction of imbalance. If the number of shares is
greater at the best ask (bid) comparing to the best bid (ask), then sell (buy) one unit of
stock. Repeat this check for each of 15 stocks and construct a corresponding portfolio.
Hold open position for one hour, then close it, redo all checks and rebalance portfolio
for the next hour.

### a. Calculate realized return on this strategy under the assumption that you can trade at midquotes (paper portfolio). What is its Sharpe ratio?